# 📌Install & Imports

In [1]:
!pip install -q --upgrade diffusers transformers accelerate safetensors gradio torch torchvision


# 📌 Load Model

In [6]:
import torch
from diffusers import DiffusionPipeline

# --- Load SDXL Turbo ---
model_id = "stabilityai/sdxl-turbo"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16"
)
pipe.to("cuda")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.35.1",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

# Create a folder to save images

In [7]:
import os
from datetime import datetime

output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)


# Generate and save function

In [8]:
from PIL import Image

def generate_images(prompt, num_images):
    results = []
    for i in range(num_images):
        image = pipe(prompt, guidance_scale=0.0, num_inference_steps=4).images[0]

        # JPEG
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        file_path = os.path.join(output_dir, f"image_{timestamp}_{i+1}.jpeg")
        image.save(file_path, "JPEG")

        results.append(file_path)
    return results


# Gradio interface

In [10]:
import gradio as gr

with gr.Blocks(css="body {background-color: #f8f9fa;} .card {padding: 15px; border-radius: 15px; background: white; box-shadow: 0px 4px 8px rgba(0,0,0,0.1);} ") as demo:
    gr.Markdown("<h1 style='text-align:center;'>🖼️ SDXL Turbo Generator</h1>")

    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(label="Enter your prompt", placeholder="Type something like: a futuristic city skyline", lines=2)
            num_images = gr.Slider(label="Number of Images", minimum=1, maximum=4, step=1, value=2)
            generate_btn = gr.Button("🚀 Generate", variant="primary")

        with gr.Column(scale=2):
            gallery = gr.Gallery(label="Generated Images", show_label=False, columns=2, height="auto")
            download_buttons = [gr.File(label=f"⬇️ Download Image {i+1}", file_types=[".jpeg"], visible=False) for i in range(4)]

    def run_generate(prompt, num_images):
        file_paths = generate_images(prompt, num_images)

        outputs = [file_paths]  # Gallery
        for i in range(4):
            if i < len(file_paths):
                outputs.append(file_paths[i])
            else:
                outputs.append(None)

        return outputs

    generate_btn.click(
        fn=run_generate,
        inputs=[prompt, num_images],
        outputs=[gallery, *download_buttons]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed44acfa831094d7ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
